In [1]:
import numpy as np
import pandas as pd
import networkx as nx
import scipy as sc
#from scipy import io
#from collections import Counter
from matplotlib import pyplot as plt
from signet.cluster import Cluster
from signet.block_models import SSBM
from sklearn.model_selection import train_test_split
from sklearn.metrics import adjusted_rand_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import plot_roc_curve
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.model_selection import train_test_split

In [2]:
df_small = pd.read_csv("Datasets/dataset_small.csv")
df_small = df_small.drop_duplicates(inplace = False)
df_small.head()

,from,to,sign
0,1,2,1
3,1,3,-1
4,1,4,-1
5,5,6,1
7,5,7,-1


In [3]:
G = nx.DiGraph()
G = nx.from_pandas_edgelist(df = df_small, source = 'from', target = 'to', edge_attr = 'sign', create_using = nx.DiGraph(), edge_key = 'sign')

In [5]:
nodes_no = nx.number_of_nodes(G)
edges_no = nx.number_of_edges(G)

In [4]:
print('number of nodes G =',nx.number_of_nodes(G) )
print('number of edges G =',nx.number_of_edges(G) )
print('Density of G:', nx.density(G))

number of nodes G = 219
number of edges G = 588
Density of G: 0.01231619957270328


In [27]:
print(G.nodes)
print(len(G.nodes))

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219]
219


In [65]:
def getSimiliarity(G, sign_tag = 'sign'):
    nodes_no = nx.number_of_nodes(G)
    sign_map = nx.get_edge_attributes(G, sign_tag)
    # similiarity_matrix = np.zeros(shape = (nodes_no, nodes_no))
    pos_matrix = np.zeros(shape = (nodes_no, nodes_no))
    neg_matrix = np.zeros(shape = (nodes_no, nodes_no))
    for u in G.nodes():
        for v in G.nodes():
            if (u != v):
                u_neighbors = set(G.adj[u])
                v_neighbors = set(G.adj[v])
                uv_neighbors = list(u_neighbors.intersection(v_neighbors))
                # similiarity_matrix[u - 1][v - 1] = np.sum([sign_map[(u, i)] * sign_map[(v, i)] for i in uv_neighbors])
                uv_similiarity = np.sum([sign_map[(u, i)] * sign_map[(v, i)] for i in uv_neighbors])
                if len(uv_neighbors) != 0:
                    uv_similiarity /= len(uv_neighbors)
                if (uv_similiarity > 0):
                    pos_matrix[u - 1][v - 1] = 1
                elif (uv_similiarity < 0):
                    neg_matrix[u - 1][v - 1] = 1
            #else:
                #similiarity_matrix[u - 1][v - 1] = 0
    pos_csc = sc.sparse.csc_matrix(pos_matrix)
    neg_csc = sc.sparse.csc_matrix(neg_matrix)
    return pos_csc, neg_csc


In [29]:
print(getSimiliarity(G, sign_tag = 'sign'))

[[ 0.  1. -1. ...  0.  0.  0.]
 [ 1.  0. -1. ...  0.  0.  0.]
 [-1. -1.  0. ...  0.  0.  0.]
 ...
 [ 0.  0.  0. ...  0.  0.  0.]
 [ 0.  0.  0. ...  0.  0.  0.]
 [ 0.  0.  0. ...  0.  0.  0.]]


In [66]:
A_p, A_n = getSimiliarity(G, sign_tag = 'sign')

In [67]:
c = Cluster((A_p, A_n))

In [107]:
def getClusters(G, k):
    spec_clus = c.spectral_cluster_bnc(k = 3, normalisation = 'sym', eigens = None, mi = None)
    clusters = []
    for j in range(k):
        clusters.append([i for i in G.nodes() if spec_clus[i - 1] == j])
    return np.array(clusters, dtype = np.ndarray)

In [109]:
clusters = getClusters(G, 3)
print(clusters[0])
print()
print(clusters[1])
print()
print(clusters[2])

[3, 4, 5, 6, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 20, 22, 23, 24, 26, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 40, 41, 42, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 55, 61, 64, 66, 67, 69, 71, 73, 75, 76, 77, 78, 79, 80, 81, 82, 83, 85, 86, 87, 88, 89, 90, 91, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 112, 114, 115, 117, 118, 119, 120, 121, 122, 123, 125, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 139, 140, 142, 144, 145, 147, 148, 149, 150, 151, 152, 153, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 176, 177, 178, 179, 181, 183, 184, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 198, 199, 201, 203, 204, 205, 206, 207, 208, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219]

[1, 2, 7, 18, 19, 21, 25, 27, 39, 43, 54, 56, 57, 58, 59, 60, 62, 63, 68, 70, 72, 74, 84, 92, 93, 94, 111, 113, 116, 124, 126, 138, 141, 143, 146, 154, 155, 174, 175, 180, 182, 185, 197, 200, 202, 209]

[65]


In [110]:
def getCommonNeighbors(G, cl1, cl2):
    x = set([])
    for i in cl1:
        x = x.union(set(G.adj[i]))
    #print('x: ', x)
    y = set([])
    for j in cl2:
        y = y.union(set(G.adj[j]))
    #print('y: ', y)
    return list(x.intersection(y))

def getClusterSimiliarity(Cluster, common_neigbors, sign_tag = 'sign'):
    sign_map = nx.get_edge_attributes(G, sign_tag)
    y = []
    for v in common_neigbors:
        x = []
        for u in Cluster:
            if (u, v) in G.edges():
                #print(str((u, v)) + ": " + str(sign_map[(u, v)]))
                x.append(sign_map[(u, v)])
        #print("#")
        y.append(np.mean(np.array(x)))
    return np.array(y)


def getSimiliarity(y1, y2):
    alpha = np.dot(y1.T, y2)
    beta = np.dot(y1.T, y1)
    gamma = np.dot(y2.T, y2)
    return alpha / np.sqrt((beta * gamma))


# main function to calculate the similiarity between two clusters
def getInterClusterSimiliarity(cl1, cl2, common_neigbors):
    y1 = getClusterSimiliarity(cl1, common_neigbors)
    y2 = getClusterSimiliarity(cl2, common_neigbors)
    return getSimiliarity(y1, y2)

# calculates similiarities between each two clusters and returns a matrix
def getAllSimiliaritiesBetweenClusters(G, clusters):
    clusters_no = clusters.shape[0]
    similiarities = np.zeros(shape = (clusters_no, clusters_no))
    for i in range(clusters_no):
        for j in range(i, clusters_no):
            common_neighbors = getCommonNeighbors(G, clusters[i], clusters[j])
            similiarities[i][j] = similiarities[j][i] = getInterClusterSimiliarity(clusters[i], clusters[j], common_neighbors)
    return similiarities

In [ ]:
def signPrediction(G, sign_map, clusters, common_neighbors, u, v, threshold):
    #if ((u, v) in G.edges()):
        #pass
    #else:
        u_c = whereIsIt(main_clusters, u)
        v_c = whereIsIt(main_clusters, v)
        S = calcX(G, v)
        S_c = whereAreThey(main_clusters, S)
        
        n = len(S)
        x = np.zeros(n)
        for i in range(n):
            x[i] = getInterClusterSimiliarity(main_clusters[u_c], main_clusters[int(S_c[i])], s_AB)
        r = np.zeros(n)
        for i in range(n):
            r[i] = AA1[(S[i], v)]
        #print(x)
        #print(r)
        result = (np.dot(x.T, r)) / (np.sum(x))
        if result > threshold:
            return 1
        else:
            return -1